In [72]:
import janitor
import pandas as pd
import numpy as np

Unpivoting(reshaping data from wide to long form) in Pandas is executed either through [pd.melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html),[pd.wide_to_long](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.wide_to_long.html), or [pd.stack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html). However, there are scenarios where a few more steps are required to massage the data into the long form we desire. Take the dataframe below, copied from [Stack Overflow](https://stackoverflow.com/questions/64061588/pandas-melt-multiple-columns-to-tabulate-a-dataset#64062002): 

In [73]:
df = pd.DataFrame(
        {
            "id": [1, 2, 3],
            "M_start_date_1": [201709, 201709, 201709],
            "M_end_date_1": [201905, 201905, 201905],
            "M_start_date_2": [202004, 202004, 202004],
            "M_end_date_2": [202005, 202005, 202005],
            "F_start_date_1": [201803, 201803, 201803],
            "F_end_date_1": [201904, 201904, 201904],
            "F_start_date_2": [201912, 201912, 201912],
            "F_end_date_2": [202007, 202007, 202007],
        }
    )

df

,id,M_start_date_1,M_end_date_1,M_start_date_2,M_end_date_2,F_start_date_1,F_end_date_1,F_start_date_2,F_end_date_2
0,1,201709,201905,202004,202005,201803,201904,201912,202007
1,2,201709,201905,202004,202005,201803,201904,201912,202007
2,3,201709,201905,202004,202005,201803,201904,201912,202007


A [beautiful solution](https://stackoverflow.com/a/64062027/7175713) was proferred : 

In [74]:
df1 = df.set_index('id')
df1.columns = df1.columns.str.split('_', expand=True)
df1 = (df1.stack(level=[0,2,3])
          .sort_index(level=[0,1], ascending=[True, False])
          .reset_index(level=[2,3], drop=True)
          .sort_index(axis=1, ascending=False)
          .rename_axis(['id','cod'])
          .reset_index())

df1

,id,cod,start,end
0,1,M,201709,201905
1,1,M,202004,202005
2,1,F,201803,201904
3,1,F,201912,202007
4,2,M,201709,201905
5,2,M,202004,202005
6,2,F,201803,201904
7,2,F,201912,202007
8,3,M,201709,201905
9,3,M,202004,202005


We propose an alternative, based on [pandas melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html), that abstracts the reshaping mechanism, allows the user to focus on the task, can be applied to other scenarios,  and is chainable : 

In [75]:
df.pivot_longer("id", names_to= ("cod", ".value"), names_pattern="(M|F)_(start|end)_.+")

,id,cod,start,end
0,1,M,201709,201905
1,1,M,202004,202005
2,1,F,201803,201904
3,1,F,201912,202007
4,2,M,201709,201905
5,2,M,202004,202005
6,2,F,201803,201904
7,2,F,201912,202007
8,3,M,201709,201905
9,3,M,202004,202005


`pivot_longer` is not a new idea; we unashamedly stole it from R's [tidyr](https://tidyr.tidyverse.org/reference/pivot_longer.html).

Do note that the `pivot_longer` function is designed primarily to work with single indexed dataframes; for MultiIndex dataframes, `pandas_melt` is more than adequate. 

`pivot_longer` can melt dataframes easily; it is just a wrapper around `pd.melt` : 

In [76]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})

df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [77]:
df.pivot_longer(index='A', column_names=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


You can dynamically select columns, using regular expressions with the `janitor.patterns` function (inspired by r's data.table's `patterns` function, and is really just a wrapper around `re.compile`), especially if it is a lot of column names, and you are *lazy* like me  😄

In [78]:
url = 'https://github.com/tidyverse/tidyr/raw/master/data-raw/billboard.csv'
df = pd.read_csv(url)

df

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2000,Yankee Grey,Another Nine Minutes,3:10,2000-04-29,86,83.0,77.0,74.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,2000,"Yearwood, Trisha",Real Live Woman,3:55,2000-04-01,85,83.0,83.0,82.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314,2000,Ying Yang Twins,Whistle While You Tw...,4:19,2000-03-18,95,94.0,91.0,85.0,84.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,2000,Zombie Nation,Kernkraft 400,3:30,2000-09-02,99,99.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
# unpivot all columns that start with 'wk'
df.pivot_longer(column_names = janitor.patterns("^(wk)"), names_to='week')

,artist,date.entered,time,track,year,week,value
0,2 Pac,2000-02-26,4:22,Baby Don't Cry (Keep...,2000,wk1,87.0
1,2 Pac,2000-02-26,4:22,Baby Don't Cry (Keep...,2000,wk2,82.0
2,2 Pac,2000-02-26,4:22,Baby Don't Cry (Keep...,2000,wk3,72.0
3,2 Pac,2000-02-26,4:22,Baby Don't Cry (Keep...,2000,wk4,77.0
4,2 Pac,2000-02-26,4:22,Baby Don't Cry (Keep...,2000,wk5,87.0
...,...,...,...,...,...,...,...
24087,matchbox twenty,2000-04-29,4:12,Bent,2000,wk72,NaN
24088,matchbox twenty,2000-04-29,4:12,Bent,2000,wk73,NaN
24089,matchbox twenty,2000-04-29,4:12,Bent,2000,wk74,NaN
24090,matchbox twenty,2000-04-29,4:12,Bent,2000,wk75,NaN


`pivot_longer` can also unpivot paired columns. Let's look at some examples from pandas `wide_to_long` docs : 

In [80]:
df = pd.DataFrame({
    'famid': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'birth': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'ht1': [2.8, 2.9, 2.2, 2, 1.8, 1.9, 2.2, 2.3, 2.1],
    'ht2': [3.4, 3.8, 2.9, 3.2, 2.8, 2.4, 3.3, 3.4, 2.9]
})

df

,famid,birth,ht1,ht2
0,1,1,2.8,3.4
1,1,2,2.9,3.8
2,1,3,2.2,2.9
3,2,1,2.0,3.2
4,2,2,1.8,2.8
5,2,3,1.9,2.4
6,3,1,2.2,3.3
7,3,2,2.3,3.4
8,3,3,2.1,2.9


In the data above, the `height` is paired with `age`. Let's see how `pivot_longer` handles this:

In [81]:
df.pivot_longer(index=['famid','birth'], names_to=('.value', 'age'), names_pattern=r"(ht)(\d)")

,famid,birth,age,ht
0,1,1,1,2.8
1,1,1,2,3.4
2,1,2,1,2.9
3,1,2,2,3.8
4,1,3,1,2.2
5,1,3,2,2.9
6,2,1,1,2.0
7,2,1,2,3.2
8,2,2,1,1.8
9,2,2,2,2.8


The first step in unpivoting a dataframe with a paired column is to specify the part of the column that will become a header, and the other part that becomes unpivoted. The `.value` symbol helps us achieve that. It indicates to the `pivot_longer` function that `ht` will be the new column name, while the rest of the data in the column(the numbers) will be unpivoted under the `age` column. How does `pivot_longer` associate `.value` to the part of the column name? The `names_pattern` or the `names_sep` arguments. For this, the `names_pattern` argument, which accepts regular expression, is a better fit; `pivot_longer` takes care of the rest.

`pd.wide_to_long` handles this already, so why bother? Let's look at another scenario where `pd.wide_to_long` would need a few more steps : 

In [107]:
#https://community.rstudio.com/t/pivot-longer-on-multiple-column-sets-pairs/43958
df = pd.DataFrame(
    {
        "off_loc": ["A", "B", "C", "D", "E", "F"],
        "pt_loc": ["G", "H", "I", "J", "K", "L"],
        "pt_lat": [
            100.07548220000001,
            75.191326,
            122.65134479999999,
            124.13553329999999,
            124.13553329999999,
            124.01028909999998,
        ],
        "off_lat": [
            121.271083,
            75.93845266,
            135.043791,
            134.51128400000002,
            134.484374,
            137.962195,
        ],
        "pt_long": [
            4.472089953,
            -144.387785,
            -40.45611048,
            -46.07156181,
            -46.07156181,
            -46.01594293,
        ],
        "off_long": [
            -7.188632000000001,
            -143.2288569,
            21.242563,
            40.937416999999996,
            40.78472,
            22.905889000000002,
        ],
    }
)

df

,off_loc,pt_loc,pt_lat,off_lat,pt_long,off_long
0,A,G,100.075482,121.271083,4.472090,-7.188632
1,B,H,75.191326,75.938453,-144.387785,-143.228857
2,C,I,122.651345,135.043791,-40.456110,21.242563
3,D,J,124.135533,134.511284,-46.071562,40.937417
4,E,K,124.135533,134.484374,-46.071562,40.784720
5,F,L,124.010289,137.962195,-46.015943,22.905889


We can unpivot with `pd.wide_to_long` by first reorganising the columns : 

In [108]:
df1 = df.copy()
df1.columns = ["_".join(col.split("_")[::-1]) for col in df1.columns]
df1.columns

Index(['loc_off', 'loc_pt', 'lat_pt', 'lat_off', 'long_pt', 'long_off'], dtype='object')

Nowe we can unpivot : 

In [109]:
pd.wide_to_long(
    df1.reset_index(),
    stubnames=["loc", "lat", "long"],
    sep="_",
    i="index",
    j="set",
    suffix="\w+",
)

37.8 ms ± 1.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Notice that we had to reset the dataframe to get an index. We can abstract all this away, using `pivot_longer` :

In [110]:
df.pivot_longer(
            names_to=["set", ".value"], names_pattern="(.+)_(.+)"
        )

14.9 ms ± 257 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Again, the key here is the `.value` symbol. `names_to` is set as ('set', '.value'), and the `names_pattern` as "(.+)_(.+)". This tells `pivot_longer` to get the `loc`, `lat`, and `long` out of the existing column names, make them new column names, while unpivoting `off` and `pt` as a new column with the header `set`. Notice that we did not have to reset the index;  `pivot_longer` allows you to focus on what you want, so you can get it and move on. 

Let's look at some more examples : 

In [111]:
 # https://stackoverflow.com/questions/45123924/convert-pandas-dataframe-from-wide-to-long/45124130
 df = pd.DataFrame([{'a_1': 2, 'ab_1': 3, 'ac_1': 4, 'a_2': 5, 'ab_2': 6, 'ac_2': 7}]
)
df

,a_1,ab_1,ac_1,a_2,ab_2,ac_2
0,2,3,4,5,6,7


The data above requires extracting `a`, `ab` and `ac` from `1` and `2`. This is another example of a paired column. We could solve this using `pd.wide_to_long`; infact there is a very good solution from [Stack Overflow](https://stackoverflow.com/a/45124775/7175713)

In [114]:
df1 = df.copy()
df1['id'] = df1.index
pd.wide_to_long(df1, ['a','ab','ac'],i='id',j='num',sep='_')

33.3 ms ± 971 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Or you could simply pass the buck to `pivot_longer` : 

In [115]:
df.pivot_longer(names_to=('.value','num'), names_sep='_')

12.2 ms ± 448 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In the solution above, we used the `names_sep`, as it is more convenient. A few more examples to get you familiar with the `.value` symbol.

In [116]:
# https://stackoverflow.com/questions/55403008/pandas-partial-melt-or-group-melt
df = pd.DataFrame([[1,1,2,3,4,5,6],[2,7,8,9,10,11,12]], 
                      columns=['id', 'ax','ay','az','bx','by','bz'])

df

,id,ax,ay,az,bx,by,bz
0,1,1,2,3,4,5,6
1,2,7,8,9,10,11,12


In [117]:
df.pivot_longer(index='id', names_to=('name','.value'), names_pattern='(.)(.)')

,id,name,x,y,z
0,1,a,1,2,3
1,1,b,4,5,6
2,2,a,7,8,9
3,2,b,10,11,12


For the code above `.value` is paired with `x`, `y`, `z`, while `a`, `b`, and `c` are unpivoted into the `name` column.

In [96]:
# https://stackoverflow.com/questions/38862832/pandas-melt-several-groups-of-columns-into-multiple-target-columns-by-name
df = pd.DataFrame({'id': [101, 102],
 'a_1': ['a', 'd'],
 'a_2': ['b', 'e'],
 'a_3': ['c', 'f'],
 'b_1': [1, 4],
 'b_2': [2, 5],
 'b_3': [3, 6],
 'c_1': ['aa', 'dd'],
 'c_2': ['bb', 'ee'],
 'c_3': ['cc', 'ff']}
)
df

,id,a_1,a_2,a_3,b_1,b_2,b_3,c_1,c_2,c_3
0,101,a,b,c,1,2,3,aa,bb,cc
1,102,d,e,f,4,5,6,dd,ee,ff


In [99]:
df.pivot_longer(index='id',names_to=('.value', 'value'), names_sep='_') # it seems single .value is not robust enough ... check and fix

,id,value,a,b,c
0,101,1,a,1,aa
1,101,2,b,2,bb
2,101,3,c,3,cc
3,102,1,d,4,dd
4,102,2,e,5,ee
5,102,3,f,6,ff


In [101]:
# https://stackoverflow.com/questions/59550804/melt-column-by-substring-of-the-columns-name-in-pandas-python
df = pd.DataFrame({'subject': [1, 2],
 'A_target_word_gd': [1, 11],
 'A_target_word_fd': [2, 12],
 'B_target_word_gd': [3, 13],
 'B_target_word_fd': [4, 14],
 'subject_type': ['mild', 'moderate']})

df

,subject,A_target_word_gd,A_target_word_fd,B_target_word_gd,B_target_word_fd,subject_type
0,1,1,2,3,4,mild
1,2,11,12,13,14,moderate


In [104]:
df.pivot_longer(index=['subject', 'subject_type'], names_to=('cond','value_type'), names_pattern='([A-Z]).*(gd|fd)')

,subject,subject_type,cond,value_type,value
0,1,mild,A,gd,1
1,1,mild,A,fd,2
2,1,mild,B,gd,3
3,1,mild,B,fd,4
4,2,moderate,A,gd,11
5,2,moderate,A,fd,12
6,2,moderate,B,gd,13
7,2,moderate,B,fd,14
